In [10]:
from bs4 import BeautifulSoup
import random
import pandas as pd
import requests
import time
import csv
import re
import sqlite3
import concurrent.futures
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.exceptions import HTTPError
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

# Define your list, or pass a list with pd.read_csv('filename.csv')

In [50]:
#passing a csv file into a list
etf_tickers = pd.read_csv('etf_tickers.csv').dropna()
etf_tickers = etf_tickers['ticker'].values.tolist()
print(etf_tickers)
len(etf_tickers)

['SPY', 'QQQ', 'IWM', 'DIA', 'IYT', 'SMH', 'EWJ', 'BBJP', 'XLK', 'XLE', 'XLY', 'XLI', 'XLF', 'XLC', 'XLV', 'XLP', 'XLU', 'XLB', 'XLRE', 'XPH', 'XBI', 'XRT', 'XHB', 'RTH', 'IYC', 'ITB', 'EWT', 'SKYY', 'CLOU', 'ROBO', 'CHIQ', 'KWEB', 'DBA', 'DBB', 'USO', 'UNG', 'UGA', 'WEAT', 'CORN', 'GLD', 'SOYB', 'CANE', 'GLDM', 'SLV', 'GDX', 'GDXJ', 'NUGT', 'JNUG', 'DUST', 'JDST', 'PPLT', 'COPX', 'UUP', 'FXY', 'FXE', 'FXB', 'FXC', 'VXX', 'UVXY', 'SVXY', 'UVIX', 'SVIX', 'BITO', 'BND', 'AGG', 'BNDX', 'LQD', 'HYG', 'JNK', 'TIP', 'VTIP', 'GOVT', 'EMB', 'IGSB', 'IEF', 'TLT', 'IGOV', 'BWX', 'FLIA', 'BWZ', 'ISHG', 'TYO', 'TBF', 'TMV', 'TBT', 'TYD', 'UBT', 'TMF', 'TQQQ', 'SQQQ', 'SH', 'SPXU', 'SPXS', 'SOXL', 'SOXS', 'TNA', 'TZA', 'FNGD', 'YINN', 'YANG', 'BOIL', 'UCO', 'SCO', 'AGQ']


104

In [45]:
# Define your list of ETF tickers defined as etf_tickers
#etf_tickers = ['EWJ', 'SPY', 'XLK']
etf_tickers = ['SCJ', 'EWJ', 'EWJV', 'FLJP']  # Add more tickers as needed

# ETF Information + holdings

In [ ]:
#DONT TOUCH THIS CODE, it works
def get_etf_info(ticker):
    try:
        #construct the ETF URL
        etf_url = f'https://etfdb.com/etf/{ticker}/#etf-ticker-profile'
        etf_ticker = etf_url.split('/')[-2]
        response = requests.get(etf_url)
        soup = BeautifulSoup(response.text, 'html.parser')


        #Basic Information
        issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
        brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
        etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
        category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
        asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()


        #weighting 
        td_elements = soup.find_all('td', class_='pull-right fs-table-alignment')
        #len(td_elements) >= 6
        sixth_element = td_elements[5]
        weighting = sixth_element.get_text(strip=True)


        expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()

        aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
        shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
        one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
        three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()

        #Dividends/Valuation
        etf_div = f'https://etfdb.com/etf/{ticker}/#etf-ticker-valuation-dividend'
        response1 = requests.get(etf_div)
        soup1 = BeautifulSoup(response1.text, 'html.parser')

        td_elements_div  = soup1.find_all('div', class_='text-center')
        pe_ratio = td_elements_div [1].get_text(strip=True)
        peer_pe_ratio = td_elements_div [3].get_text(strip=True)
        
        #Div Yield/Divvy
        divvy  = soup1.find_all('td', class_='center', attrs={'data-th':'Fund'})

        
        #Performance
        etf_perf_url  = f'https://etfdb.com/etf/{ticker}/#performance'
        response2 = requests.get(etf_perf_url)
        soup2 = BeautifulSoup(response2.text, 'html.parser')
        
#         etf_perf = soup2.find_all('td', {'data-th': 'Fund'}) 
#         etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
        performance = soup2.find_all('td', class_='col-lg-4 col-xs-4')


# print(issuer, brand, etf_url, category, asset_class_size, weighting, one_mo_avg_volume, three_mo_avg_volume, expense_ratio, sep="\n")
# Create a dictionary with variable names and their values
        data_dict = {
            'ETF Ticker': etf_ticker,
            'Issuer': issuer,
            'Brand': brand,
            'ETF Home Page': etf_home_page,
            'Category': category,
            'Asset Class Size': asset_class_size,
            'Weighting': weighting,
            'Expense Ratio': expense_ratio,

            'PE Ratio' : pe_ratio,
            'Peer PE Ratio' : peer_pe_ratio,
            'ETF Dividend' : divvy[8].get_text(strip=True),
            'ETF Div Yield' : divvy[11].get_text(strip=True),
            'ETF Peer Dividend' : divvy[10].get_text(strip=True),
            'ETF Peer Div Yield' : divvy[11].get_text(strip=True),   

            #perf
            '1 Month Return': performance[0].get_text(strip=True),
            '3 Month Return': performance[3].get_text(strip=True),
            'YTD Return': performance[6].get_text(strip=True),
            '1 Year Return': performance[9].get_text(strip=True),
            '3 Year Return': performance[12].get_text(strip=True),
            '5 Year Return': performance[15].get_text(strip=True),

            'Peer 1 Month Return': performance[1].get_text(strip=True),
            'Peer 3 Month Return': performance[4].get_text(strip=True),
            'Peer YTD Return': performance[7].get_text(strip=True),
            'Peer 1 Year Return': performance[10].get_text(strip=True),
            'Peer 3 Year Return': performance[13].get_text(strip=True),
            'Peer 5 Year Return': performance[16].get_text(strip=True),

            'AUM': aum,
            'Shares float': shares_float,
            '1 Month Avg. Volume': one_mo_avg_volume,
            '3 Month Avg. Volume': three_mo_avg_volume,

        }

        return data_dict
    except AttributeError as attr_err:
        print(f"Attribute Error for ticker {ticker}: {attr_err}")
    except HTTPError as http_err:
        if http_err.response.status_code == 404:
            print(f"Skipping ticker {ticker}: Ticker not found.")
        else:
            print(f"Skipping ticker {ticker}: HTTP Error - {http_err}")
    except Exception as ex:
        print(f"Skipping ticker {ticker}: An error occurred - {ex}")
    return None



#ETF Holdings code
def etf_holdings(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3  # Number of retries
    for _ in range(retries):
        try:
            response = requests.get(etf_holdings_url)
            response.raise_for_status()  # Raise exception for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')

            etf_name = etf_holdings_url.split('/')[-2]

            rows = soup.find_all('tr', {'class': 'holding'})

            stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
            stock_names = soup.find_all('td', {'data-th': 'Holding'})
            stock_weights = soup.find_all('td', {'data-th': '% Assets'})

            stock_symbols = [item.get_text() for item in stock_symbols]
            stock_names = [item.get_text() for item in stock_names]
            stock_weights = [item.get_text() for item in stock_weights]

            data = []

            for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                data.append([name, symbol, weight])

            df = pd.DataFrame(data, columns=["holdings", "symbol", "% assets"])  
            return df                                                            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            time.sleep(random.uniform(0.0, 3.0))  # Adding a random sleep between retries

    return None  # Return None if scraping fails after retries


etf_data_dict = {}  # Dictionary to store ETF data
not_scraped = []  # List to store tickers that were not successfully scraped




#Execution for etf_holdings
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the etf_holdings function for each ticker
    futures = [executor.submit(etf_holdings, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for ticker, future in zip(etf_tickers, as_completed(futures)):
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        else:
            not_scraped.append(ticker)
        pbar.update(1)

        
# Create a DataFrame containing data from scraped ETFs
combined_data = pd.concat(etf_data_dict.values(), keys=etf_data_dict.keys(), names=['Ticker'])

if not_scraped:
    print("Tickers that were not successfully scraped:")
    print(not_scraped)

print("Data saved.")
print(combined_data)


results = []

#Execution for get_etf_info
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the get_etf_info function for each ticker
    futures = [executor.submit(get_etf_info, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)
        pbar.update(1)
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(results)

100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


Data saved.
                                        holdings symbol % assets
Ticker                                                          
SCJ    0                      Toyota Motor Corp.   7203    5.03%
       1                  Sony Group Corporation   6758    2.74%
       2    Mitsubishi UFJ Financial Group, Inc.   8306    2.30%
       3                     Keyence Corporation   6861    2.06%
       4                     Tokyo Electron Ltd.   8035    1.62%
       5               Shin-Etsu Chemical Co Ltd   4063    1.52%
       6   Sumitomo Mitsui Financial Group, Inc.   8316    1.45%
       7                            Hitachi,Ltd.   6501    1.43%
       8                  Mitsubishi Corporation   8058    1.43%
       9                        Mitsui & Co.,Ltd   8031    1.41%
       10        Daiichi Sankyo Company, Limited   4568    1.36%
       11                     Itochu Corporation   8001    1.28%
       12                DAIKIN INDUSTRIES, LTD.   6367    1.27%
       13    

100%|██████████| 4/4 [00:12<00:00,  3.02s/it]


In [ ]:
# Display the DataFrame
df

,ETF Ticker,Issuer,Brand,ETF Home Page,Category,Asset Class Size,Weighting,Expense Ratio,PE Ratio,Peer PE Ratio,ETF Dividend,ETF Div Yield,ETF Peer Dividend,ETF Peer Div Yield,1 Month Return,3 Month Return,YTD Return,1 Year Return,3 Year Return,5 Year Return,Peer 1 Month Return,Peer 3 Month Return,Peer YTD Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
0,FLJP,Franklin Templeton,Franklin,https://www.franklintempleton.com/investments/...,Japan Equities,Large-Cap,Market Cap,0.09%,13.56,10.40,$ 0.36,2.10%,$ 0.57,2.10%,N/A,3.32%,13.38%,10.35%,4.86%,3.18%,1.71%,7.32%,19.23%,17.63%,10.27%,4.90%,"$1,554.6 M",57.6 M,"429,221","460,256"
1,EWJV,BlackRock Financial Management,iShares,https://www.ishares.com/us/products/307263/ish...,Japan Equities,Large-Cap,Market Cap,0.15%,9.08,10.40,$ 0.39,2.35%,$ 0.66,2.35%,1.68%,6.64%,15.78%,16.35%,10.71%,N/A,1.71%,7.32%,19.23%,17.63%,10.27%,4.90%,$217.2 M,7.8 M,"107,904","96,239"
2,SCJ,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Japan Equities,Mid-Cap,Market Cap,0.50%,12.66,10.40,$ 0.50,0.72%,$ 0.50,0.72%,1.09%,0.83%,7.95%,7.95%,2.39%,0.27%,1.71%,7.32%,19.23%,17.63%,10.27%,4.90%,$118.2 M,1.7 M,"36,067","55,352"
3,EWJ,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Japan Equities,Large-Cap,Market Cap,0.50%,12.72,10.40,$ 0.43,0.90%,$ 0.56,0.90%,-0.29%,3.43%,13.50%,10.02%,4.42%,2.94%,1.71%,7.32%,19.23%,17.63%,10.27%,4.90%,"$13,396.8 M",218.3 M,"7,315,813","7,540,452"


In [ ]:
# Transpose data
df.T

,0
ETF Ticker,SCJ
Issuer,BlackRock Financial Management
Brand,iShares
ETF Home Page,http://us.ishares.com/product_info/fund/overvi...
Category,Japan Equities
Asset Class Size,Mid-Cap
Weighting,Market Cap
Expense Ratio,0.50%
PE Ratio,12.66
Peer PE Ratio,10.40


In [ ]:
#Save to csv
df.T.to_csv('etf_info.csv', index=True)

# Search in dataframe 'database' 

In [ ]:
# Display the stored data for a specific ETF
selected_etf = 'SCJ'   #change this
if selected_etf in etf_data_dict:
    print(f"Holdings data for {selected_etf}:")
    print(etf_data_dict[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

Holdings data for SCJ:
                                 holdings symbol % assets
0                      Toyota Motor Corp.   7203    5.03%
1                  Sony Group Corporation   6758    2.74%
2    Mitsubishi UFJ Financial Group, Inc.   8306    2.30%
3                     Keyence Corporation   6861    2.06%
4                     Tokyo Electron Ltd.   8035    1.62%
5               Shin-Etsu Chemical Co Ltd   4063    1.52%
6   Sumitomo Mitsui Financial Group, Inc.   8316    1.45%
7                            Hitachi,Ltd.   6501    1.43%
8                  Mitsubishi Corporation   8058    1.43%
9                        Mitsui & Co.,Ltd   8031    1.41%
10        Daiichi Sankyo Company, Limited   4568    1.36%
11                     Itochu Corporation   8001    1.28%
12                DAIKIN INDUSTRIES, LTD.   6367    1.27%
13                  Honda Motor Co., Ltd.   7267    1.26%
14                   SoftBank Group Corp.   9984    1.16%


# Make it look nicer

In [ ]:
# Group the data by ETF
grouped_data = combined_data.groupby('Ticker')

# Iterate through each ETF group and display the data
for etf, group in grouped_data:
    print(f"Holdings data for {etf}:")
    print(group)
    print('\n')

Holdings data for EWJ:
                                        holdings symbol % assets
Ticker                                                          
EWJ    0                      Toyota Motor Corp.   7203    9.78%
       1    Mitsubishi UFJ Financial Group, Inc.   8306    4.88%
       2                  Mitsubishi Corporation   8058    3.40%
       3   Sumitomo Mitsui Financial Group, Inc.   8316    3.20%
       4                   Honda Motor Co., Ltd.   7267    2.68%
       5          Takeda Pharmaceutical Co. Ltd.   4502    2.65%
       6                        KDDI Corporation   9433    2.38%
       7             Tokio Marine Holdings, Inc.   8766    2.17%
       8            Mizuho Financial Group, Inc.   8411    2.15%
       9                            Hitachi,Ltd.   6501    2.12%
       10                         SoftBank Corp.   9434    1.76%
       11                           Komatsu Ltd.   6301    1.43%
       12                       Mitsui & Co.,Ltd   8031    1.42%
  

# Iterate through every ETF and output into 1 csv, named after the ETF. I am sure there is a better way to do this to manipulate the df and output into a single excel, with each tab named after it. This is the hacky way

In [ ]:
# Store the ETF data in a database or file
for ticker, df in etf_data_dict.items():
    df.to_csv(f'{ticker}_holdings.csv', index=False)

    
# Get a list of the created CSV files
csv_files = [f'{ticker}_holdings.csv' for ticker in etf_data_dict.keys()]
csv_files

['SCJ_holdings.csv',
 'EWJ_holdings.csv',
 'EWJV_holdings.csv',
 'FLJP_holdings.csv']

# Creates 1 excel file, with each tab containing the holdings of each ETF

In [ ]:
# List of CSV files
# csv_files = [
#     'AGQ_holdings.csv', 'BBJP_holdings.csv', 'BITO_holdings.csv', 'BND_holdings.csv', 'BNDX_holdings.csv',
#     'BOIL_holdings.csv', 'BWZ_holdings.csv', 'BWX_holdings.csv', 'CANE_holdings.csv', 'CHIQ_holdings.csv',
#     'CLOU_holdings.csv', 'COPX_holdings.csv', 'CORN_holdings.csv', 'DBA_holdings.csv', 'DBB_holdings.csv',
#     'DIA_holdings.csv', 'DUST_holdings.csv', 'EWT_holdings.csv', 'EWJ_holdings.csv', 'FLIA_holdings.csv',
#     'FNGD_holdings.csv', 'FXB_holdings.csv', 'FXC_holdings.csv', 'FXE_holdings.csv', 'FXY_holdings.csv',
#     'GDX_holdings.csv', 'GDXJ_holdings.csv', 'GLD_holdings.csv', 'GLDM_holdings.csv', 'GOVT_holdings.csv',
#     'HYG_holdings.csv', 'IEF_holdings.csv', 'IGOV_holdings.csv', 'IGSB_holdings.csv', 'ISHG_holdings.csv',
#     'ITB_holdings.csv',  'IWM_holdings.csv', 'IYC_holdings.csv',  'JDST_holdings.csv',
#     'JNK_holdings.csv', 'JNUG_holdings.csv', 'KWEB_holdings.csv', 'LQD_holdings.csv', 'NUGT_holdings.csv',
#     'PPLT_holdings.csv', 'QQQ_holdings.csv', 'RTH_holdings.csv', 'SCO_holdings.csv', 'SH_holdings.csv',
#     'SKYY_holdings.csv', 'SLV_holdings.csv', 'SMH_holdings.csv', 'SOXL_holdings.csv', 'SOXS_holdings.csv',
#     'SOYB_holdings.csv', 'SPXU_holdings.csv', 'SPXS_holdings.csv', 'SPY_holdings.csv', 'SQQQ_holdings.csv',
#     'SVIX_holdings.csv', 'SVXY_holdings.csv', 'TBF_holdings.csv', 'TBT_holdings.csv', 'TIP_holdings.csv',
#     'TLT_holdings.csv', 'TMF_holdings.csv', 'TMV_holdings.csv', 'TNA_holdings.csv', 'TQQQ_holdings.csv',
#     'TZA_holdings.csv', 'UBT_holdings.csv', 'UCO_holdings.csv', 'UGA_holdings.csv', 'UNG_holdings.csv',
#     'USO_holdings.csv', 'UUP_holdings.csv', 'UVIX_holdings.csv', 'UVXY_holdings.csv', 'VTIP_holdings.csv',
#     'VXX_holdings.csv', 'WEAT_holdings.csv', 'XBI_holdings.csv', 'XHB_holdings.csv', 'XLB_holdings.csv',
#     'XLC_holdings.csv', 'XLE_holdings.csv', 'XLF_holdings.csv', 'XLI_holdings.csv', 'XLK_holdings.csv',
#     'XLP_holdings.csv', 'XLRE_holdings.csv', 'XLU_holdings.csv', 'XLV_holdings.csv', 'XLY_holdings.csv',
#     'XPH_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv',
#     'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv',
#     'YINN_holdings.csv', 'YANG_holdings.csv'
# ]


# Create a Pandas Excel writer using XlsxWriter as the engine
excel_writer = pd.ExcelWriter('burry.xlsx', engine='xlsxwriter')

# Loop through each CSV file and write its contents to a new sheet
for csv_file in csv_files:
    # Read CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Get the ETF name from the CSV filename
    etf_name = csv_file.split('_')[0]
    
    # Write the DataFrame to a new sheet with the ETF name
    df.to_excel(excel_writer, sheet_name=etf_name, index=False)

# Save the Excel workbook
excel_writer.save()

print('Merged CSV files into a single Excel file.')

Merged CSV files into a single Excel file.


C:\Users\Jonat\AppData\Local\Temp\ipykernel_36820\4239240233.py:43: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


# Improve, to save into a SQLite database

In [ ]:
'''
# Store the ETF data in an SQLite3 database
db_filename = 'etf_holdings.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create a table to store ETF data
create_table_query = '''
CREATE TABLE IF NOT EXISTS etf_holdings (
    id INTEGER PRIMARY KEY,
    ticker TEXT,
    holdings TEXT,
    symbol TEXT,
    percent_assets TEXT
)
'''
cursor.execute(create_table_query)

# Insert data into the table
for ticker, df in etf_data_dict.items():
    for _, row in df.iterrows():
        insert_query = '''
        INSERT INTO etf_holdings (ticker, holdings, symbol, percent_assets)
        VALUES (?, ?, ?, ?)
        '''
        cursor.execute(insert_query, (ticker, row['holdings'], row['symbol'], row['% assets']))

# Commit changes and close the connection
conn.commit()
conn.close()

# Display the stored data for a specific ETF
selected_etf = 'EWJ'
conn = sqlite3.connect(db_filename)
query = f'SELECT * FROM etf_holdings WHERE ticker = "{selected_etf}"'
selected_df = pd.read_sql_query(query, conn)
conn.close()

if not selected_df.empty:
    print(f"Holdings data for {selected_etf}:")
    print(selected_df)
else:
    print(f"No data found for {selected_etf}.")
    
'''